# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [5]:
# Write your code below.
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [6]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [14]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files

['../../05_src/data/prices\\A\\A_2000\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2016\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2017\\part.0.pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [25]:
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1),Adj_Close_lag_1 = x['Adj Close'].shift(1))
)
dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)
dd_feat


C:\Users\H511855\AppData\Local\Temp\ipykernel_42220\3274437315.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
npartitions=13078,,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [40]:
df = dd_feat.compute()
df

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
Ticker,,,,,,,,,,,,
DOV,2005-01-03 00:00:00+00:00,18.734230,27.604422,28.227835,27.544090,28.107174,762305.0,2005,NaN,NaN,NaN,0.683744
DOV,2005-01-04 00:00:00+00:00,18.647797,27.477058,28.013327,27.403320,27.865852,1330378.0,2005,27.604422,18.734230,-0.004614,0.610006
DOV,2005-01-05 00:00:00+00:00,18.283840,26.940788,27.785412,26.907272,27.477058,1176575.0,2005,27.477058,18.647797,-0.019517,0.878139
DOV,2005-01-06 00:00:00+00:00,18.110964,26.686062,26.981009,26.625732,26.981009,1395122.0,2005,26.940788,18.283840,-0.009455,0.355276
DOV,2005-01-07 00:00:00+00:00,17.847105,26.297268,26.726282,26.277157,26.625732,1192984.0,2005,26.686062,18.110964,-0.014569,0.449125
...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2023-12-22 00:00:00+00:00,44.080002,44.080002,44.790001,43.730000,44.070000,813915.0,2023,43.900002,43.900002,0.004100,1.060001
CTLT,2023-12-26 00:00:00+00:00,45.250000,45.250000,45.259998,44.189999,44.189999,642878.0,2023,44.080002,44.080002,0.026543,1.070000
CTLT,2023-12-27 00:00:00+00:00,45.500000,45.500000,45.840000,45.173000,45.470001,1803542.0,2023,45.250000,45.250000,0.005525,0.667000


In [35]:
df['MA'] = df['Returns'].rolling(10).mean()
df


Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,MA
Ticker,,,,,,,,,,,,,
DOV,2008-01-02 00:00:00+00:00,21.197414,29.776308,30.935989,29.769606,30.721481,1504321.0,2008,NaN,NaN,NaN,1.166384,NaN
DOV,2008-01-03 00:00:00+00:00,21.135386,29.689165,30.077959,29.628836,29.870155,1533262.0,2008,29.776308,21.197414,-0.002927,0.449123,NaN
DOV,2008-01-04 00:00:00+00:00,20.290728,28.502670,29.581911,28.502670,29.467955,2272444.0,2008,29.689165,21.135386,-0.039964,1.079241,NaN
DOV,2008-01-07 00:00:00+00:00,19.660818,27.617828,28.824432,27.383211,28.797619,4208190.0,2008,28.502670,20.290728,-0.031044,1.441221,NaN
DOV,2008-01-08 00:00:00+00:00,18.415314,25.868252,27.751896,25.774405,27.691565,3965327.0,2008,27.617828,19.660818,-0.063350,1.977491,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
CTLT,2011-12-23 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN,NaN
CTLT,2011-12-27 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN,NaN
CTLT,2011-12-28 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,2011,NaN,NaN,NaN,NaN,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
- Yes.
+ Would it have been better to do it in Dask? Why?
- No, Panda loads data into memory as a single DataFrame which make the process easier and more efficient.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.